In [ ]:
from bs4 import BeautifulSoup
import csv
import os
import shutil

## --- parameter ---
CONTEST = "BW131" # Contest name
CONTEST_URL = "https://leetcode.com/contest/biweekly-contest-131/" # Contest URL
LANG = ".java" # the file extension gonna deal with
OUTPUT_HTML = CONTEST+"/Q4_java.html" # the output HTML generated by the detection tool
THRESHOLD = 95 # Merge only if Both A is THRESHOLD% similar to B, and B is THRESHOLD% similar to B

In [ ]:
def Manual_add(user1, user2, cheater):
    group_1 = -1;
    group_2 = -1;
    
    for j, group in enumerate(cheater):
        if user1 in group:
            group_1 = j
        if user2 in group:
            group_2 = j
    
    if group_1 == -1 and group_2 == -1:
        cheater.append([user1, user2])
    elif group_1 == -1:
        cheater[group_2].append(user1)
    elif group_2 == -1:
        cheater[group_1].append(user2)
    elif group_1 != group_2:
        cheater[group_1].extend(cheater[group_2])
        del cheater[group_2]

In [ ]:
with open(OUTPUT_HTML, encoding="utf-8") as f:
    soup = BeautifulSoup(f.read(), 'html.parser')

In [ ]:
## --- parameter ---
# The cheater group found so far

cheater = [
[1, 3, 7],
[13, 73]
]

In [ ]:
# It will output the cheater list. You can copy the result to the code block above to record it

## --- parameter ---
IGNORE = [916, 1373] # user to ignore, i.e. maybe you know they're not cheating or exclude temporary
# Manual_add(13, 73, cheater) # Manually merge cheater

for row in soup.find_all("tr"):
    i = row.find_all("i")

    user1 = int(i[0].get_text()[13:-len(LANG)])
    user2 = int(i[1].get_text()[13:-len(LANG)])

    if user1 in IGNORE or user2 in IGNORE:
        continue

    b = row.find_all("b")
    p1 = float(b[0].get_text()[:-1])
    p2 = float(b[1].get_text()[:-1])

    group_1 = -1;
    group_2 = -2;
    
    for j, group in enumerate(cheater):
        if user1 in group:
            group_1 = j
        if user2 in group:
            group_2 = j

    if p1 >= THRESHOLD and p2 >= THRESHOLD:
        if group_1 == -1 and group_2 == -2:
            cheater.append([user1, user2])
        elif group_1 == -1:
            cheater[group_2].append(user1)
        elif group_2 == -2:
            cheater[group_1].append(user2)
        elif group_1 != group_2:
            cheater[group_1].extend(cheater[group_2])
            del cheater[group_2]
    elif group_1 != group_2:
        print(f"Test file: {i[0].get_text()} ({b[0].get_text()})\nReference file: {i[1].get_text()} ({b[1].get_text()})")

print("[")
for i in cheater:
    i.sort()
    print(i, ",")
print("]")

In [ ]:
# # clear 100%
# QUESTION = '/Q3/'

# for group in cheater:
#     for idx in range(1, len(group)):
#         folder = str(group[idx]//1000)+"001-"+str(group[idx]//1000+1)+"000/"
        
#         if os.path.exists(CONTEST+QUESTION+folder+str(group[idx])+LANG):
#             os.remove(CONTEST+QUESTION+folder+str(group[idx])+LANG)

In [ ]:
# # Not necessary. Copy to test for further testing the first file of each group
# QUESTION = '/Q4/'

# try:
#     shutil.rmtree(CONTEST+"/test")
# except:
#     pass
# try:
#     os.mkdir(CONTEST+"/test")
# except:
#     pass

# for group in cheater:
#     folder = str(group[0]//1000)+"001-"+str(group[0]//1000+1)+"000/"

#     shutil.copyfile(CONTEST+QUESTION+folder+str(group[0])+LANG, CONTEST+"/test/"+str(group[0])+LANG)

In [ ]:
## generate markdown, and record the rank, user name, and submission ID of the mass cheating group to csv

## --- parameter ---
Q = 4 # i.e. 4 means Q4
CSV = CONTEST+"/Q4_cpp.csv" # the name of csv gonna be created to record the mass cheating group
# the mass cheating group
cheater = [13, 73]

print("how many cheater: ", len(cheater))

with open(CONTEST+"/"+CONTEST+".csv") as f:
    rows = list(csv.reader(f))
    
    with open(CSV) as csv_:
        for i in cheater:
            print(str(i)+"," + rows[i][0]+"," + rows[i][Q], file=csv_)
    
    with open("markdown.md", "w") as md:
        print("| rank | user | Q"+str(Q)+"   |\n| ---- | ---- | ---- |", file=md)
    
        for i in cheater:
            print("|", i, "| ["+rows[i][0]+"](https://leetcode.com/u/"+rows[i][0]+") |", CONTEST_URL+"submissions/detail/"+rows[i][Q], "|", file=md)